In [1]:
import specufex
import sys
import os 
import obspy
from specufex import BayesianNonparametricNMF
import h5py
import yaml
import numpy as np
import time
from tqdm import trange
## change this to input arg
yamlPath = "/Users/theresasawi/Documents/11_Manuscripts/Methods_Paper/data/yaml/demo_150Hz_v3.yaml"
# yamlPath = "/Users/theresasawi/Documents/11_Manuscripts/Methods_Paper/data/yaml/demo_150Hz_dec5.yaml"


In [2]:


####################################################################################
####################################################################################
###
### Load yaml file settings, creat paths, set parameters
###
####################################################################################
####################################################################################



with open(yamlPath) as stream:
    config = yaml.safe_load(stream)
    
path_config = config["paths"]
key = path_config["key"]
print("Project key:", key)



projectPath = path_config["projectPath"]
SpecUFEx_H5_name = 'SpecUFEx_' + path_config["h5name"] #f'SpecUFEx_{key}.hdf5'
SpecUFEx_H5_path = projectPath + 'data/H5files/' + SpecUFEx_H5_name


#SpecUFEx parameters
specufex_config = config['specufexParams']
N_patterns_NMF = specufex_config['N_patterns_NMF']               
nmf_batchsz = specufex_config['nmf_batchsz']               
nmf_nbatch = specufex_config['nmf_nbatch']
N_states_HMM = specufex_config['N_states_HMM']
hmm_batchsz = specufex_config['hmm_batchsz']        
hmm_nbatch = specufex_config['hmm_nbatch']        

Project key: demo_50Hz_v3


In [3]:
####################################################################################
####################################################################################
###
### Stack spectrograms for SpecUFEx input
###
####################################################################################
####################################################################################

X = []

with h5py.File(SpecUFEx_H5_path,'a') as fileLoad:
    for evID in fileLoad['spectrograms']:
#         print(evID)
        specMat = fileLoad['spectrograms'].get(evID)[:]
        X.append(specMat)

    X = np.array(X)

# ================
print(np.shape(X))





(1496, 200, 1950)


In [4]:
specMat.shape

(200, 1950)

In [5]:
####################################################################################
####################################################################################
###
### Run SpecUFEx!
###
####################################################################################
####################################################################################
nmf = specufex.BayesianNonparametricNMF(X.shape)


start_time = time.time() # 3.0922142305639055 hours for 45k sgrams


print('fitting NMF')
nmf.fit(X, verbose=0)

end_time = time.time()

elapsed_time_NMF_fit = end_time - start_time
####################################################################################
####################################################################################


start_time = time.time() # 3.0922142305639055 hours for 45k sgrams

print('transforming NMF')    
Vs = nmf.transform(X)

end_time = time.time()

elapsed_time_NMF_transform = end_time - start_time
####################################################################################
####################################################################################


start_time = time.time() # 3.0922142305639055 hours for 45k sgrams

hmm = specufex.BayesianHMM(nmf.num_pat, nmf.gain)


print('fitting HMM')
hmm.fit(Vs)

end_time = time.time()

elapsed_time_HMM_fit = end_time - start_time
####################################################################################
####################################################################################


start_time = time.time() # 3.0922142305639055 hours for 45k sgrams

print('transforming HMM')    
fingerprints, As, gams = hmm.transform(Vs)

end_time = time.time()

elapsed_time_HMM_transform = end_time - start_time
        

fitting NMF
transforming NMF
fitting HMM
transforming HMM


In [6]:
print(f"Elapsed time: {elapsed_time_NMF_fit} seconds")
print(f"Elapsed time: {elapsed_time_NMF_transform} seconds")
print(f"Elapsed time: {elapsed_time_HMM_fit} seconds")
print(f"Elapsed time: {elapsed_time_HMM_transform} seconds")
print('total time',(elapsed_time_NMF_fit+elapsed_time_NMF_transform+elapsed_time_HMM_fit+elapsed_time_HMM_transform)/3600, 'hours')

##demo50Hz: Elapsed time: 680.7948398590088 seconds
# Elapsed time: 206.7134461402893 seconds
# Elapsed time: 743.1099891662598 seconds
# Elapsed time: 682.7004978656769 seconds
# total time 0.6425885480642318 hours

##demo250Hz_v2: 

#(1815, 1020, 475)

# demo_150Hz_dec20
# (1496, 610, 475)
# Elapsed time: 2724.646586894989 seconds
# Elapsed time: 939.3281400203705 seconds
# Elapsed time: 748.2971248626709 seconds
# Elapsed time: 685.5354537963867 seconds
# total time 1.4160575848817825 hours

# demo_150Hz_v3
# Elapsed time: 26845.914207935333 seconds
# Elapsed time: 3255.791141986847 seconds
# Elapsed time: 2295.416862010956 seconds
# Elapsed time: 2253.0588159561157 seconds
# total time 9.625050285524793 hours


Elapsed time: 3635.334984064102 seconds
Elapsed time: 798.50861120224 seconds
Elapsed time: 2451.43989109993 seconds
Elapsed time: 2498.375291109085 seconds
total time 2.6065718826320436 hours


In [7]:
# ####################################################################################
# ####################################################################################
# ###
# ### Run SpecUFEx! Nate's version
# ###
# ####################################################################################
# ####################################################################################



# nmf = BayesianNonparametricNMF(X.shape,num_pat=N_patterns_NMF)

# t = trange(nmf_nbatch, desc="NMF fit progress ", leave=True)
# for i in t:
#     idx = np.random.randint(len(X), size=nmf_batchsz)
#     nmf.fit(X)
#     t.set_postfix_str(f"Patterns: {nmf.num_pat}")
    

# Vs = nmf.transform(X)

# hmm = BayesianHMM(nmf.num_pat, nmf.gain, num_state=N_states_HMM, Neff=50000)



# t = trange(hmm_nbatch, desc="HMM fit progress ", leave=True)
# for i in t:
#     idx = np.random.randint(Vs.shape[0], size=hmm_batchsz)
#     hmm.fit(Vs[idx])

# fingerprints, As, gams = hmm.transform(Vs)

In [8]:
####################################################################################
####################################################################################
###
### save output to H5
###
####################################################################################
####################################################################################

print('writing all output to h5')
with h5py.File(SpecUFEx_H5_path,'a') as fileLoad:


    ##fingerprints are top folder
    if 'fingerprints' in fileLoad.keys():
        del fileLoad["fingerprints"]
    fp_group = fileLoad.create_group('fingerprints')

    if 'SpecUFEX_output' in fileLoad.keys():
        del fileLoad["SpecUFEX_output"]
    out_group = fileLoad.create_group("SpecUFEX_output")

    # write fingerprints: ===============================
    for i, evID in enumerate(fileLoad['spectrograms']):
        fp_group.create_dataset(name= evID, data=fingerprints[i])


    # write the SpecUFEx out: ===========================
    # maybe include these, but they are not yet tested.
    ACM_group = fileLoad.create_group("SpecUFEX_output/ACM")
    STM_group = fileLoad.create_group("SpecUFEX_output/STM")

    for i, evID in enumerate(fileLoad['spectrograms']):
        ACM_group.create_dataset(name=evID,data=Vs[i]) #ACM
        STM_group.create_dataset(name=evID,data=gams[i]) #STM

    gain_group = fileLoad.create_group("SpecUFEX_output/ACM_gain")
    W_group                      = fileLoad.create_group("SpecUFEX_output/W")
    EB_group                     = fileLoad.create_group("SpecUFEX_output/EB")
    ## # # delete probably ! gain_group                   = fileLoad.create_group("SpecUFEX_output/gain")
    #RMM_group                    = fileLoad.create_group("SpecUFEX_output/RMM")

    W_group.create_dataset(name='W',data=nmf.EW)
    EB_group.create_dataset(name=evID,data=hmm.EB)
    gain_group.create_dataset(name='gain',data=nmf.gain) #same for all data
    # RMM_group.create_dataset(name=evID,data=RMM)
    






writing all output to h5


In [9]:
os.system("say 'SpecUFEx complete'")    


0